In [55]:
#-------------SOURCE LIBRARIES AND CODE-----
#source('cb_config.R')
#source('cb_utilities.R')
#source('cb_strategy_specific_functions.R')
#source('general_ssp_utilities.R')

from general_ssp_utilities import *
from cb_config import *
from cb_utilities import cb_calculate_system_costs, cb_calculate_transformation_costs,cb_process_interactions
from data_reader import CBFilesReader
from cb_config import * 
import pickle

from typing import List
import pandas as pd 
import os 
import re 
import numpy as np
from new_strategies import strategy_list, strategy_dict

In [56]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
def build_path(PATH : List[str]) -> str:
    return os.path.abspath(os.path.join(*PATH))

#-------------READ THE DATA-----------------
## ----------- DEFINE PATHS
CB_REFAC_PATH = build_path([os.getcwd(), ".."])
CB_REFAC_DATA_PATH = build_path([CB_REFAC_PATH, "data"])
STRATEGY_SPECIFIC_CB_PATH_FILES = build_path([CB_REFAC_DATA_PATH, "strategy_specific_cb_files"])
#SSP_RESULTS_DATA_PATH = build_path([CB_REFAC_DATA_PATH, "ssp_model_results"])
SSP_RESULTS_DATA_PATH = build_path([CB_REFAC_DATA_PATH, "ssp_model_results", "iran"])
DEFINITION_FILES_PATH = build_path([CB_REFAC_DATA_PATH, "definition_files"])
STRATEGY_DEFINITION_FILE_PATH = build_path([SSP_RESULTS_DATA_PATH, "ATTRIBUTE_STRATEGY_new.csv"])

OUTPUT_PATH = build_path([CB_REFAC_PATH, "output"])

#data_filename = build_path([SSP_RESULTS_DATA_PATH, "sisepuede_results_sisepuede_run_ssp.csv"] )
#data_filename = build_path([SSP_RESULTS_DATA_PATH, "sisepuede_results_sisepuede_run_iran.csv"] )
data_filename = build_path([SSP_RESULTS_DATA_PATH, "iran_new.csv"] )
primary_filename = build_path([SSP_RESULTS_DATA_PATH, "ATTRIBUTE_PRIMARY_new.csv"])
strategy_filename = build_path([SSP_RESULTS_DATA_PATH, "ATTRIBUTE_STRATEGY_new.csv"])


output_file = pd.read_csv(data_filename)

In [58]:
output_file.head()

,primary_id,region,time_period,area_agrc_crops_bevs_and_spices,area_agrc_crops_cereals,area_agrc_crops_fibers,area_agrc_crops_fruits,area_agrc_crops_herbs_and_other_perennial_crops,area_agrc_crops_nuts,area_agrc_crops_other_annual,...,yf_agrc_fruits_tonne_ha,yf_agrc_herbs_and_other_perennial_crops_tonne_ha,yf_agrc_nuts_tonne_ha,yf_agrc_other_annual_tonne_ha,yf_agrc_other_woody_perennial_tonne_ha,yf_agrc_pulses_tonne_ha,yf_agrc_rice_tonne_ha,yf_agrc_sugar_cane_tonne_ha,yf_agrc_tubers_tonne_ha,yf_agrc_vegetables_and_vines_tonne_ha
0,0,iran,7,2.354342e+06,5.236903e+06,1.086620e+06,2.832254e+06,3.654765e+06,1.994651e+06,2.638574e+06,...,12.291287,3.021928,1.207774,1.122455,1.862022,2.213576,3.954868,76.33122,25.149791,13.691656
1,0,iran,8,2.379435e+06,5.292717e+06,1.098201e+06,2.862440e+06,3.693717e+06,2.015910e+06,2.666696e+06,...,12.291287,3.021928,1.207774,1.122455,1.862022,2.213576,3.954868,76.33122,25.149791,13.691656
2,0,iran,9,2.404280e+06,5.347981e+06,1.109668e+06,2.892328e+06,3.732284e+06,2.036959e+06,2.694540e+06,...,12.291287,3.021928,1.207774,1.122455,1.862022,2.213576,3.954868,76.33122,25.149791,13.691656
3,0,iran,10,2.428879e+06,5.402700e+06,1.121021e+06,2.921921e+06,3.770472e+06,2.057801e+06,2.722110e+06,...,12.291287,3.021928,1.207774,1.122455,1.862022,2.213576,3.954868,76.33122,25.149791,13.691656
4,0,iran,11,2.453237e+06,5.456879e+06,1.132263e+06,2.951223e+06,3.808283e+06,2.078437e+06,2.749407e+06,...,12.291287,3.021928,1.207774,1.122455,1.862022,2.213576,3.954868,76.33122,25.149791,13.691656


In [59]:
#-------------PREPARE THE DATA--------------
#Read data
data = output_file.copy()
data.head()

,primary_id,region,time_period,area_agrc_crops_bevs_and_spices,area_agrc_crops_cereals,area_agrc_crops_fibers,area_agrc_crops_fruits,area_agrc_crops_herbs_and_other_perennial_crops,area_agrc_crops_nuts,area_agrc_crops_other_annual,...,yf_agrc_fruits_tonne_ha,yf_agrc_herbs_and_other_perennial_crops_tonne_ha,yf_agrc_nuts_tonne_ha,yf_agrc_other_annual_tonne_ha,yf_agrc_other_woody_perennial_tonne_ha,yf_agrc_pulses_tonne_ha,yf_agrc_rice_tonne_ha,yf_agrc_sugar_cane_tonne_ha,yf_agrc_tubers_tonne_ha,yf_agrc_vegetables_and_vines_tonne_ha
0,0,iran,7,2.354342e+06,5.236903e+06,1.086620e+06,2.832254e+06,3.654765e+06,1.994651e+06,2.638574e+06,...,12.291287,3.021928,1.207774,1.122455,1.862022,2.213576,3.954868,76.33122,25.149791,13.691656
1,0,iran,8,2.379435e+06,5.292717e+06,1.098201e+06,2.862440e+06,3.693717e+06,2.015910e+06,2.666696e+06,...,12.291287,3.021928,1.207774,1.122455,1.862022,2.213576,3.954868,76.33122,25.149791,13.691656
2,0,iran,9,2.404280e+06,5.347981e+06,1.109668e+06,2.892328e+06,3.732284e+06,2.036959e+06,2.694540e+06,...,12.291287,3.021928,1.207774,1.122455,1.862022,2.213576,3.954868,76.33122,25.149791,13.691656
3,0,iran,10,2.428879e+06,5.402700e+06,1.121021e+06,2.921921e+06,3.770472e+06,2.057801e+06,2.722110e+06,...,12.291287,3.021928,1.207774,1.122455,1.862022,2.213576,3.954868,76.33122,25.149791,13.691656
4,0,iran,11,2.453237e+06,5.456879e+06,1.132263e+06,2.951223e+06,3.808283e+06,2.078437e+06,2.749407e+06,...,12.291287,3.021928,1.207774,1.122455,1.862022,2.213576,3.954868,76.33122,25.149791,13.691656


In [60]:
#Merge model output with strategy attributes (mainly the strategy_code)
run_attributes = ssp_merge_run_attributes(primary_filename, strategy_filename)
run_attributes.head()
merged_data = run_attributes[['primary_id', 'strategy_code', 'future_id']].merge(right = data, on='primary_id')
data = merged_data.copy()


In [ ]:

### Convertimos BASE ---> LNDU:PLUR
data.loc[data.strategy_code=='BASE', "strategy_code"]="LNDU:PLUR"

#clean the data of furnace gas and crude
temp_data_cols = data.columns

cols_to_keep = [string for string in temp_data_cols if not re.match(re.compile('totalvalue.*furnace_gas'), string)]
cols_to_keep = [string for string in cols_to_keep if not re.match(re.compile('totalvalue_.*_fuel_consumed_.*_fuel_crude'), string)]
cols_to_keep = [string for string in cols_to_keep if not re.match(re.compile('totalvalue_.*_fuel_consumed_.*_fuel_electricity'), string)] #10.13 ADDED THIS SO SECTOR FUELS EXCLUDE ELECTRICITY

data = data[cols_to_keep]

#add calculation of total TLUs to data
TLU_CONVERSION_FILE_PATH = build_path([STRATEGY_SPECIFIC_CB_PATH_FILES, "lvst_tlu_conversions.csv"]) 
tlu_conversions = pd.read_csv(TLU_CONVERSION_FILE_PATH)

pop_livestock = data[SSP_GLOBAL_SIMULATION_IDENTIFIERS + [i for i in cols_to_keep if "pop_lvst" in i]]
pop_livestock = pop_livestock.melt(id_vars=['primary_id', 'time_period', 'region', 'strategy_code', 'future_id'])
pop_livestock = pop_livestock.merge(right=tlu_conversions, on = "variable")

pop_livestock["total_tlu"] = pop_livestock["value"] * pop_livestock["TLU"]

pop_livestock_summarized = pop_livestock.groupby(SSP_GLOBAL_SIMULATION_IDENTIFIERS).\
                                            agg({"total_tlu" : sum}).\
                                            rename(columns={"total_tlu":"lvst_total_tlu"}).\
                                            reset_index()

data = data.merge(right = pop_livestock_summarized, on = SSP_GLOBAL_SIMULATION_IDENTIFIERS)

#replace any lagging references to "PFLO:SOCIOTECHNICAL" with "PFLO:CHANGE_CONSUMPTION"
data.loc[data["strategy_code"] == "PFLO:SOCIOTECHNICAL", "strategy_code"] = "PFLO:CHANGE_CONSUMPTION"
SSP_GLOBAL_list_of_strategies = data["strategy_code"].unique()
SSP_GLOBAL_list_of_variables = list(set(data.columns) - set(SSP_GLOBAL_SIMULATION_IDENTIFIERS))

#-------------REMOVE BASE FOR COST BENEFIT ANALYSIS------------
data = data.query("strategy_code!='BASE'").reset_index(drop=True)

#data[""]{i:j for i,j in pd.read_csv(primary_filename)[["primary_id", "strategy_id"]].to_records(index = False)}
#-------------READ DEFINITION FILES IN AND CALCULATE COSTS AND BENEFITS----------

#maps strategies to transformations, from James
#This file tells us which transformation in is in each strategy
strategy2tx = pd.read_csv(build_path([DEFINITION_FILES_PATH, 'attribute_strategy_code.csv']))

## Agregamos las estrategias 6003 : 'PFLO:CONSTRAINED', 6004 : 'PFLO:TECHNOLOGICAL_ADOPTION',6005 :'PFLO:UNCONSTRAINED'
#STRATEGY_DEFINITION_FILE_PATH = build_path([SSP_RESULTS_DATA_PATH, "strategy_definitions.csv"])
ssp_strategy_definition_james = pd.read_csv(STRATEGY_DEFINITION_FILE_PATH)

actualiza_claves_tx  = {'TX:AGRC:EXPAND_CONSERVATION_AGRICULTURE' : 'TX:AGRC:INC_CONSERVATION_AGRICULTURE',
 #'TX:AGRC:INC_RESIDUE_REMOVAL' : ,
 #'TX:AGRC_DEC_DEMAND_FOR_UNHEALTHY_CROPS',
 'TX:CCSQ:INCREASE_CAPTURE' : 'TX:CCSQ:INC_CAPTURE',
 #'TX:ENTC:LEAST_COST',
 'TX:ENTC:TARGET_RENEWABLE_ENERGY_PRODUCTION' : 'TX:ENTC:TARGET_RENEWABLE_ELEC',
 #'TX:GNRL:DEC_RED_MEAT_CONSUMPTION',
 'TX:INEN:FUEL_SWITCH_HI_AND_LO_HEAT' : 'TX:INEN:SHIFT_FUEL_HEAT',
 #'TX:INEN:FUEL_SWITCH_HI_HEAT',
 #'TX:INEN:FUEL_SWITCH_LO_HEAT',
 'TX:LNDU:DEC_DEFORESTATION_AND_INC_SILVOPASTURE' : 'TX:LNDU:DEC_DEFORESTATION',
  'TX:LNDU:DEC_DEFORESTATION' : 'TX:LNDU:DEC_DEFORESTATION',
 #'TX:LNDU:INC_LAND_REHABILITIATION',
 'TX:PFLO:IND_INC_CCS' : 'TX:PFLO:INC_IND_CCS',
 'TX:SCOE:FUEL_SWITCH_HEAT' : 'TX:SCOE:SHIFT_FUEL_HEAT',
 'TX:SOIL:DEC_LIME_APPLIED' : 'TX:SOIL:DEC_LIME_APPLIED_HIGHEST',
 'TX:SOIL:DEC_N_APPLIED' : 'TX:SOIL:DEC_N_APPLIED_HIGHEST',
 'TX:TRNS:FUEL_SWITCH_LIGHT_DUTY' : 'TX:TRNS:SHIFT_FUEL_LIGHT_DUTY',
 'TX:TRNS:FUEL_SWITCH_MARITIME' : 'TX:TRNS:SHIFT_FUEL_MARITIME',
 'TX:TRNS:FUEL_SWITCH_MEDIUM_DUTY' : 'TX:TRNS:SHIFT_FUEL_MEDIUM_DUTY',
 'TX:TRNS:FUEL_SWITCH_RAIL' : 'TX:TRNS:SHIFT_FUEL_RAIL',
 'TX:TRNS:INC_EFFICIENCY' : 'TX:TRNS:INC_EFFICIENCY_NON_ELECTRIC',
 'TX:TRNS:INC_OCCUPANCY' : 'TX:TRNS:INC_OCCUPANCY_LIGHT_DUTY',
 'TX:TRNS:MODE_SHIFT_FREIGHT' : 'TX:TRNS:SHIFT_MODE_FREIGHT',
 'TX:TRNS:MODE_SHIFT_PASSENGER' : 'TX:TRNS:SHIFT_MODE_PASSENGER',
 'TX:TRNS:MODE_SHIFT_REGIONAL' : 'TX:TRNS:SHIFT_MODE_REGIONAL'}
 
actualiza_claves_tx = {j:i for i,j in actualiza_claves_tx.items()}

def quita_sufijo(tx_nombre : str) -> str:
    for sufijo in ["_LOW","_LOWEST","_HIGHEST","_HIGHER", "_HIGH", "_LOWER","_FROMTECH", "_URBPLAN"]:
        if tx_nombre.endswith(sufijo):
            tx_nombre = tx_nombre.replace(sufijo, "")

    return tx_nombre


# strategy_list = [
# 'PFLO:CONSTRAINED',
# 'PFLO:UNCONSTRAINED',
# 'PFLO:NET_ZERO',
# ]

for id_tx in strategy_list: # VERY IMPORTANT!!! Allows you to add more strategies
    tx_in_strat = ssp_strategy_definition_james.set_index("strategy_code").loc[id_tx, "transformation_specification"].split("|")
    df_new_strat_name = pd.DataFrame({"strategy_code" : [id_tx]})
    df_binary_tx = pd.DataFrame([[1]*len(tx_in_strat)] , columns=tx_in_strat)
    df_new_strat = pd.concat([df_new_strat_name, df_binary_tx], axis = 1)
    df_new_strat.columns = [quita_sufijo(i) for i in df_new_strat.columns]

    df_new_strat = df_new_strat.rename(columns = actualiza_claves_tx)

    columnas_coinciden = list(set(strategy2tx.columns).intersection(df_new_strat.columns))
    strategy2tx = pd.concat([strategy2tx, df_new_strat[columnas_coinciden]], ignore_index = True)

strategy2tx = strategy2tx.replace(np.nan, 0)

/tmp/ipykernel_122231/3122185991.py:24: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  agg({"total_tlu" : sum}).\


In [62]:
strategy2tx.head()

,strategy_code,TX:AGRC:DEC_CH4_RICE,TX:AGRC_DEC_DEMAND_FOR_UNHEALTHY_CROPS,TX:AGRC:DEC_EXPORTS,TX:AGRC:DEC_LOSSES_SUPPLY_CHAIN,TX:AGRC:EXPAND_CONSERVATION_AGRICULTURE,TX:AGRC:INC_PRODUCTIVITY,TX:AGRC:INC_RESIDUE_REMOVAL,TX:CCSQ:INCREASE_CAPTURE,TX:ENTC:DEC_LOSSES,...,TX:WALI:INC_TREATMENT_INDUSTRIAL,TX:WALI:INC_TREATMENT_RURAL,TX:WALI:INC_TREATMENT_URBAN,TX:WASO:DEC_CONSUMER_FOOD_WASTE,TX:WASO:INC_ANAEROBIC_AND_COMPOST,TX:WASO:INC_CAPTURE_BIOGAS,TX:WASO:INC_ENERGY_FROM_BIOGAS,TX:WASO:INC_ENERGY_FROM_INCINERATION,TX:WASO:INC_LANDFILLING,TX:WASO:INC_RECYCLING
0,BASE,0,0.0,0.0,0,0,0,0.0,0.0,0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0
1,AGRC:DEC_CH4_RICE,1,0.0,0.0,0,0,0,0.0,0.0,0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0
2,AGRC:DEC_EXPORTS,0,0.0,1.0,0,0,0,0.0,0.0,0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0
3,AGRC:DEC_LOSSES_SUPPLY_CHAIN,0,0.0,0.0,1,0,0,0.0,0.0,0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0
4,AGRC:DEC_LOSSES_SUPPLY_CHAIN_PLUR,0,0.0,0.0,1,0,0,0.0,0.0,0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0


In [63]:
strategy2tx.tail()

,strategy_code,TX:AGRC:DEC_CH4_RICE,TX:AGRC_DEC_DEMAND_FOR_UNHEALTHY_CROPS,TX:AGRC:DEC_EXPORTS,TX:AGRC:DEC_LOSSES_SUPPLY_CHAIN,TX:AGRC:EXPAND_CONSERVATION_AGRICULTURE,TX:AGRC:INC_PRODUCTIVITY,TX:AGRC:INC_RESIDUE_REMOVAL,TX:CCSQ:INCREASE_CAPTURE,TX:ENTC:DEC_LOSSES,...,TX:WALI:INC_TREATMENT_INDUSTRIAL,TX:WALI:INC_TREATMENT_RURAL,TX:WALI:INC_TREATMENT_URBAN,TX:WASO:DEC_CONSUMER_FOOD_WASTE,TX:WASO:INC_ANAEROBIC_AND_COMPOST,TX:WASO:INC_CAPTURE_BIOGAS,TX:WASO:INC_ENERGY_FROM_BIOGAS,TX:WASO:INC_ENERGY_FROM_INCINERATION,TX:WASO:INC_LANDFILLING,TX:WASO:INC_RECYCLING
127,PFLO:ALL_NO_STOPPING_DEFORESTATION_PLUR,1,1.0,0.0,1,1,1,0.0,0.0,1,...,1,1,1,1,1.0,1.0,1.0,1.0,1.0,1
128,PFLO:ALL_PLUR,1,1.0,0.0,1,1,1,0.0,0.0,1,...,1,1,1,1,1.0,1.0,1.0,1.0,1.0,1
129,PFLO:CONSTRAINED,1,0.0,0.0,1,1,1,0.0,0.0,1,...,1,1,1,1,0.0,0.0,0.0,0.0,0.0,1
130,PFLO:UNCONSTRAINED,1,0.0,0.0,1,1,1,0.0,1.0,1,...,1,1,1,1,1.0,1.0,1.0,1.0,1.0,1
131,PFLO:NET_ZERO,1,0.0,1.0,1,1,1,0.0,1.0,1,...,1,1,1,1,1.0,1.0,1.0,1.0,1.0,1


In [64]:
strategy2tx.strategy_code.unique()

array(['BASE', 'AGRC:DEC_CH4_RICE', 'AGRC:DEC_EXPORTS',
       'AGRC:DEC_LOSSES_SUPPLY_CHAIN',
       'AGRC:DEC_LOSSES_SUPPLY_CHAIN_PLUR',
       'AGRC:EXPAND_CONSERVATION_AGRICULTURE', 'AGRC:INC_RESIDUE_REMOVAL',
       'AGRC:INC_PRODUCTIVITY', 'AGRC:INC_PRODUCTIVITY_PLUR', 'AGRC:ALL',
       'AGRC:ALL_PLUR', 'LNDU:DEC_DEFORESTATION',
       'LNDU:INC_LAND_REHABILITIATION', 'LNDU:INC_SILVOPASTURE',
       'LNDU:INC_SILVOPASTURE_PLUR', 'LNDU:INC_SUSTAINABLE_GRAZING',
       'LNDU:PLUR', 'LSMM:INC_MANAGEMENT_CATTLE_PIGS',
       'LSMM:INC_MANAGEMENT_OTHER', 'LSMM:INC_MANAGEMENT_POULTRY',
       'LSMM:INC_CAPTURE_BIOGAS', 'LSMM:BUNDLE_INC_MANAGEMENT',
       'LVST:INC_PRODUCTIVITY', 'LVST:INC_PRODUCTIVITY_PLUR',
       'LVST:DEC_ENTERIC_FERMENTATION', 'LVST:DEC_EXPORTS', 'LVST:ALL',
       'LVST:ALL_PLUR', 'SOIL:DEC_N_APPLIED', 'SOIL:DEC_LIME_APPLIED',
       'AF:ALL', 'AF:ALL_NO_STOPPING_DEFORESTATION_PLUR', 'AF:ALL_PLUR',
       'TRWW:INC_CAPTURE_BIOGAS', 'TRWW:INC_COMPLIANCE_SEPTIC',


In [ ]:
"""
nuevas_strategy2tx = []

mapp_id2codename = {
     6003 : 'PFLO:CONSTRAINED', 
     6004 : 'PFLO:TECHNOLOGICAL_ADOPTION',
     6005 :'PFLO:UNCONSTRAINED'
}

for id_tx in [6003, 6004, 6005]:
    nuevas_strategy2tx.append(
        [mapp_id2codename[id_tx]] + [int(i in ssp_strategy_definition_james.set_index("strategy_id").loc[id_tx].transformation_specification.split("|")) for i in strategy2tx.columns[1:] ]
    )

nuevas_strategy2tx = pd.DataFrame(nuevas_strategy2tx, columns=strategy2tx.columns)
strategy2tx = pd.concat([strategy2tx, nuevas_strategy2tx], ignore_index = True)

"""

# strategy_dict = {
#     6003 : 'PFLO:CONSTRAINED', 
#     6004 : 'PFLO:UNCONSTRAINED',
#     6005 :'PFLO:NET_ZERO',
# }

#tells us which strategies to evaluate costs and benefit iffor
mapp_id2codename = strategy_dict # VERY IMPORTANT!!! Allows you to add more strategies


strategy_cost_instructions = pd.read_csv(build_path([DEFINITION_FILES_PATH, 'strategy_cost_instructions.csv']))

new_strategy_cost_instructions = pd.DataFrame(
    [
       [mapp_id2codename[id_tx]]*2 + ["LNDU:PLUR",1,1]  for id_tx in mapp_id2codename.keys()
    ],
    columns = ['strategy', 'strategy_code', 'comparison_code', 'evaluate_system_costs','evaluate_transformation_costs']
)

strategy_cost_instructions = pd.concat([strategy_cost_instructions, new_strategy_cost_instructions], ignore_index=True)

#the list of all the cost factor files in the system, and the functions they should be evaluated with
cost_factor_names = pd.read_csv(build_path([DEFINITION_FILES_PATH, 'system_cost_factors_list.csv']))  

#defines how each transformation is evaluated, including difference variables, cost multipliers, etc.
transformation_cost_definitions = pd.read_csv(build_path([DEFINITION_FILES_PATH, 'transformation_cost_definitions.csv']), encoding="latin")

#calculate system costs
cb_data = CBFilesReader(CB_REFAC_DATA_PATH)


In [66]:
data.head()

,primary_id,strategy_code,future_id,region,time_period,area_agrc_crops_bevs_and_spices,area_agrc_crops_cereals,area_agrc_crops_fibers,area_agrc_crops_fruits,area_agrc_crops_herbs_and_other_perennial_crops,...,yf_agrc_herbs_and_other_perennial_crops_tonne_ha,yf_agrc_nuts_tonne_ha,yf_agrc_other_annual_tonne_ha,yf_agrc_other_woody_perennial_tonne_ha,yf_agrc_pulses_tonne_ha,yf_agrc_rice_tonne_ha,yf_agrc_sugar_cane_tonne_ha,yf_agrc_tubers_tonne_ha,yf_agrc_vegetables_and_vines_tonne_ha,lvst_total_tlu
0,0,LNDU:PLUR,0,iran,7,2.354342e+06,5.236903e+06,1.086620e+06,2.832254e+06,3.654765e+06,...,3.021928,1.207774,1.122455,1.862022,2.213576,3.954868,76.33122,25.149791,13.691656,1.690368e+07
1,0,LNDU:PLUR,0,iran,8,2.379435e+06,5.292717e+06,1.098201e+06,2.862440e+06,3.693717e+06,...,3.021928,1.207774,1.122455,1.862022,2.213576,3.954868,76.33122,25.149791,13.691656,1.683926e+07
2,0,LNDU:PLUR,0,iran,9,2.404280e+06,5.347981e+06,1.109668e+06,2.892328e+06,3.732284e+06,...,3.021928,1.207774,1.122455,1.862022,2.213576,3.954868,76.33122,25.149791,13.691656,1.677561e+07
3,0,LNDU:PLUR,0,iran,10,2.428879e+06,5.402700e+06,1.121021e+06,2.921921e+06,3.770472e+06,...,3.021928,1.207774,1.122455,1.862022,2.213576,3.954868,76.33122,25.149791,13.691656,1.671272e+07
4,0,LNDU:PLUR,0,iran,11,2.453237e+06,5.456879e+06,1.132263e+06,2.951223e+06,3.808283e+06,...,3.021928,1.207774,1.122455,1.862022,2.213576,3.954868,76.33122,25.149791,13.691656,1.665060e+07


In [67]:
data.strategy_code.unique()

array(['LNDU:PLUR', 'PFLO:CONSTRAINED', 'PFLO:UNCONSTRAINED',
       'PFLO:NET_ZERO'], dtype=object)

In [68]:
strategy_cost_instructions.tail()

,strategy,strategy_code,comparison_code,evaluate_system_costs,evaluate_transformation_costs
62,LVST: All transformations,LVST:ALL,LNDU:PLUR,1,1
63,LNDU: Plur,LNDU:PLUR,LNDU:PLUR,0,0
64,PFLO:CONSTRAINED,PFLO:CONSTRAINED,LNDU:PLUR,1,1
65,PFLO:UNCONSTRAINED,PFLO:UNCONSTRAINED,LNDU:PLUR,1,1
66,PFLO:NET_ZERO,PFLO:NET_ZERO,LNDU:PLUR,1,1


In [69]:
# Strategies not mapped in strategy_cost_instructions.csv
missing_str_list = set(data.strategy_code.unique()) - set(strategy_cost_instructions.strategy_code.unique())

df_missing_strategy_list = pd.DataFrame(list(missing_str_list), columns=['strat_name'])
df_missing_strategy_list.sort_values(by='strat_name', ascending=True, inplace=True)
df_missing_strategy_list.to_csv('missing_strategies.csv', index=False)
df_missing_strategy_list.head()

,strat_name


In [70]:
cost_factor_names.head()

,system_cost_filename,include,cb_function
0,entc_production_cost_factors.csv,1,cb_apply_cost_factors
1,enfu_fuel_cost_factors.csv,1,cb_system_fuel_costs
2,enfu_sector_fuel_cost_factors.csv,1,cb_apply_cost_factors
3,entc_air_pollution_cost_factors.csv,1,cb_apply_cost_factors
4,trns_air_pollution_cost_factors.csv,1,cb_apply_cost_factors


In [71]:
cb_data

In [72]:
SSP_GLOBAL_list_of_variables

['frac_scoe_heat_energy_residential_hydrocarbon_gas_liquids',
 'nemomod_entc_discounted_operating_costs_pp_oil',
 'qty_waso_landfilled_yard_tonne',
 'ef_enfu_combustion_tonne_co2_per_tj_fuel_solar',
 'emission_co2e_ch4_trww_untreated_with_sewerage_treatment',
 'yf_agrc_other_annual_tonne_ha',
 'pij_lndu_grasslands_to_settlements',
 'vehicle_distance_traveled_trns_rail_passenger_natural_gas',
 'exports_enfu_pj_fuel_diesel',
 'qty_trww_cod_removed_treated_latrine_improved_tonne',
 'area_lndu_conversion_from_settlements_to_settlements',
 'emission_co2e_ch4_lsmm_paddock_pasture_range',
 'emission_co2e_n2o_trww_untreated_no_sewerage_effluent',
 'ef_lndu_conv_forests_mangroves_to_forests_mangroves_gg_co2_ha',
 'emission_co2e_ch4_inen_agriculture_and_livestock',
 'frac_inen_energy_recycled_wood_diesel',
 'vol_trww_ww_treated_latrine_improved_m3',
 'frac_ccsq_heat_energy_direct_air_capture_hydrogen',
 'nemomod_entc_capital_cost_pp_wind_mm_usd_per_gw',
 'physparam_waso_k_paper',
 'energy_demand

In [73]:
SSP_GLOBAL_list_of_strategies

array(['LNDU:PLUR', 'PFLO:CONSTRAINED', 'PFLO:UNCONSTRAINED',
       'PFLO:NET_ZERO'], dtype=object)

In [74]:
print(
    """
    ++++++++++++++++++++++++++++++++++++++++++++++++++++
    
                    CALCULATE SYSTEM COST

    +++++++++++++++++++++++++++++++++++++++++++++++++++++
    """
)
results_system = cb_calculate_system_costs(data, strategy_cost_instructions, cost_factor_names, cb_data, SSP_GLOBAL_list_of_variables, SSP_GLOBAL_list_of_strategies)



    ++++++++++++++++++++++++++++++++++++++++++++++++++++
    
                    CALCULATE SYSTEM COST

    +++++++++++++++++++++++++++++++++++++++++++++++++++++
    
============Evaluating system costs for PFLO:CONSTRAINED =================
Cost Factor Loop: PFLO:CONSTRAINED vs LNDU:PLUR for cost factor afolu_crop_livestock_production_cost_factors and cost factor function cb_apply_cost_factors
---------Costs for: cb:lvst:lvst_value:livestock_produced:buffalo
                       pop_lvst_buffalo
DESDE cb_difference_between_two_strategies
pop_lvst_buffalo
Valor multiplicador
260.0
---------Costs for: cb:lvst:lvst_value:livestock_produced:cattle_dairy
                       pop_lvst_cattle_dairy
DESDE cb_difference_between_two_strategies
pop_lvst_cattle_dairy
Valor multiplicador
260.0
---------Costs for: cb:lvst:lvst_value:livestock_produced:cattle_nondairy
                       pop_lvst_cattle_nondairy
DESDE cb_difference_between_two_strategies
pop_lvst_cattle_nondairy
Valor multi

In [75]:
results_system

,strategy_code,future_id,region,time_period,difference_variable,difference_value,variable,value
0,PFLO:CONSTRAINED,0,iran,7,pop_lvst_buffalo,0.000000e+00,cb:lvst:lvst_value:livestock_produced:buffalo,0.000000e+00
1,PFLO:CONSTRAINED,0,iran,8,pop_lvst_buffalo,0.000000e+00,cb:lvst:lvst_value:livestock_produced:buffalo,0.000000e+00
2,PFLO:CONSTRAINED,0,iran,9,pop_lvst_buffalo,0.000000e+00,cb:lvst:lvst_value:livestock_produced:buffalo,0.000000e+00
3,PFLO:CONSTRAINED,0,iran,10,pop_lvst_buffalo,3.600000e+01,cb:lvst:lvst_value:livestock_produced:buffalo,9.360000e+03
4,PFLO:CONSTRAINED,0,iran,11,pop_lvst_buffalo,5.400000e+01,cb:lvst:lvst_value:livestock_produced:buffalo,1.404000e+04
...,...,...,...,...,...,...,...,...
17569,PFLO:NET_ZERO,0,iran,31,gasrecovered_waso_biogas_landfills_tonne,1.073504e+07,cb:waso:fuel_cost:waste_to_energy_value:landfill,3.473102e+09
17570,PFLO:NET_ZERO,0,iran,32,gasrecovered_waso_biogas_landfills_tonne,1.180684e+07,cb:waso:fuel_cost:waste_to_energy_value:landfill,3.819861e+09
17571,PFLO:NET_ZERO,0,iran,33,gasrecovered_waso_biogas_landfills_tonne,1.294905e+07,cb:waso:fuel_cost:waste_to_energy_value:landfill,4.189398e+09
17572,PFLO:NET_ZERO,0,iran,34,gasrecovered_waso_biogas_landfills_tonne,1.416799e+07,cb:waso:fuel_cost:waste_to_energy_value:landfill,4.583763e+09


In [76]:
df_strategies = pd.read_csv(strategy_filename)

# Split each row by "|" and flatten the list of lists
all_values = df_strategies.transformation_specification.str.split('|').explode()

# Get the unique values as a list
unique_transformations = all_values.unique().tolist()

# Print the unique values
print(unique_transformations)

['TX:BASE', 'TX:AGRC:DEC_CH4_RICE', 'TX:AGRC:DEC_EXPORTS', 'TX:AGRC:DEC_LOSSES_SUPPLY_CHAIN', 'TX:AGRC:INC_CONSERVATION_AGRICULTURE', 'TX:AGRC:INC_PRODUCTIVITY', 'TX:LNDU:DEC_DEFORESTATION', 'TX:LNDU:DEC_SOC_LOSS_PASTURES', 'TX:LNDU:INC_REFORESTATION', 'TX:LNDU:INC_SILVOPASTURE', 'TX:LNDU:PLUR', 'TX:LSMM:INC_CAPTURE_BIOGAS', 'TX:LSMM:INC_MANAGEMENT_CATTLE_PIGS', 'TX:LSMM:INC_MANAGEMENT_OTHER', 'TX:LSMM:INC_MANAGEMENT_POULTRY', 'TX:LVST:DEC_ENTERIC_FERMENTATION', 'TX:LVST:DEC_EXPORTS', 'TX:LVST:INC_PRODUCTIVITY', 'TX:SOIL:DEC_LIME_APPLIED', 'TX:SOIL:DEC_N_APPLIED', 'TX:TRWW:INC_CAPTURE_BIOGAS', 'TX:TRWW:INC_COMPLIANCE_SEPTIC', 'TX:WALI:INC_TREATMENT_INDUSTRIAL', 'TX:WALI:INC_TREATMENT_RURAL', 'TX:WALI:INC_TREATMENT_URBAN', 'TX:WASO:DEC_CONSUMER_FOOD_WASTE', 'TX:WASO:INC_ANAEROBIC_AND_COMPOST', 'TX:WASO:INC_CAPTURE_BIOGAS', 'TX:WASO:INC_ENERGY_FROM_BIOGAS', 'TX:WASO:INC_ENERGY_FROM_INCINERATION', 'TX:WASO:INC_LANDFILLING', 'TX:WASO:INC_RECYCLING', 'TX:CCSQ:INC_CAPTURE', 'TX:ENTC:DEC_LOSS

In [77]:
missing_transformation_list = set(unique_transformations) - set(transformation_cost_definitions.transformation_code)

In [78]:
df_missing_transformation_list = pd.DataFrame(list(missing_transformation_list), columns=['trans_name'])
df_missing_transformation_list.sort_values(by='trans_name', ascending=True, inplace=True)
df_missing_transformation_list.to_csv('missing_transformations.csv', index=False)
df_missing_transformation_list.head()

,trans_name
61,TX:AGRC:DEC_CH4_RICE_LOWER
58,TX:AGRC:DEC_CH4_RICE_LOWEST
93,TX:AGRC:DEC_EXPORTS
21,TX:AGRC:DEC_LOSSES_SUPPLY_CHAIN_LOW
57,TX:AGRC:INC_CONSERVATION_AGRICULTURE


In [79]:
#calcualte transformation costs

print(
    """
    ++++++++++++++++++++++++++++++++++++++++++++++++++++
    
                    CALCULATE TRANSFORMATION COST

    +++++++++++++++++++++++++++++++++++++++++++++++++++++
    """
)

results_tx = cb_calculate_transformation_costs(data, 
                                  strategy_cost_instructions,
                                  strategy2tx, 
                                  transformation_cost_definitions, 
                                  cb_data,
                                  SSP_GLOBAL_list_of_variables, 
                                  SSP_GLOBAL_list_of_strategies)

RESULTS_SYSTEMS_PATH = build_path([OUTPUT_PATH, "python", "results_system_python.csv"])
RESULTS_TX_SYSTEMS_PATH = build_path([OUTPUT_PATH, "python", "results_tx_python.csv"])



    ++++++++++++++++++++++++++++++++++++++++++++++++++++
    
                    CALCULATE TRANSFORMATION COST

    +++++++++++++++++++++++++++++++++++++++++++++++++++++
    
masking 
 0     False
1     False
2     False
3     False
4     False
5     False
6      True
7      True
8      True
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17     True
18     True
19    False
20    False
21     True
22     True
23    False
24    False
25     True
26     True
27     True
28     True
29    False
30     True
31     True
32     True
33    False
34    False
35     True
36     True
37     True
38     True
39     True
40    False
41    False
42    False
43    False
44    False
45    False
46     True
47     True
48    False
Name: transformation_code, dtype: bool
The following transformations are in strategy: PFLO:CONSTRAINED
0: TX:AGRC:DEC_CH4_RICE
1: TX:AGRC:DEC_LOSSES_SUPPLY_CHAIN
2: TX:AGRC:EXPAND_CONSERVATION_AGRICULTURE
3: TX:AGRC:INC_PRODU

/home/tony/ssp/cb_ssp_refac/src/cb_strategy_specific_functions.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp["difference_variable"] = diff_var_name
/home/tony/ssp/cb_ssp_refac/src/cb_strategy_specific_functions.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp["difference_value"] = data_merged["difference"]
/home/tony/ssp/cb_ssp_refac/src/cb_strategy_specific_functions.py:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

                       emission_co2e_n2o_ippu_production_paper
APLICANDO CB_FRACTION_CHANGE
invert : 1.0
frac_var : ef_ippu_tonne_n2o_per_tonne_production_electronics
strategy_code_tx : PFLO:NET_ZERO
strategy_code_base : LNDU:PLUR
diff_var : emission_co2e_n2o_ippu_production_paper
                       emission_co2e_n2o_ippu_production_other_product_manufacturing
APLICANDO CB_FRACTION_CHANGE
invert : 1.0
frac_var : ef_ippu_tonne_n2o_per_tonne_production_electronics
strategy_code_tx : PFLO:NET_ZERO
strategy_code_base : LNDU:PLUR
diff_var : emission_co2e_n2o_ippu_production_other_product_manufacturing
============Evaluating transformation costs for PFLO:NET_ZERO-cb:ippu:technical_cost:abating_gases:hfcs
Usaremos la función específica a la estrategia cb_fraction_change
                       emission_co2e_hfcs_ippu_product_use_product_use_ods_other
APLICANDO CB_FRACTION_CHANGE
invert : 1.0
frac_var : ef_ippu_tonne_sf6_per_tonne_production_electronics
strategy_code_tx : PFLO:NET_ZERO
stra

In [ ]:
#combine the results
results_all = pd.concat([results_system, results_tx], ignore_index = True)

SSP_GLOBAL_list_of_cbvars = results_all["variable"].unique()


#-------------POST PROCESS SIMULATION RESULTS---------------
#Post process interactions among strategies that affect the same variables
postprocess_interactions = pd.read_csv(build_path([DEFINITION_FILES_PATH, 'strategy_interaction_definitions.csv']))

results_all_pp = cb_process_interactions(results_all, strategy2tx, postprocess_interactions)

#POST PROCESS TRANSPORT CB ISSUES (These should be addressed properly in the next round of analysis)
#congestion and safety benefits should be 0 in strategies that only make fuel efficiency gains and fuel switching

cond_transport_cb_issues_better_base = (results_all_pp["strategy_code"]=="PFLO:BETTER_BASE") & (results_all_pp["variable"].apply(lambda x : any(k in x for k in ["congestion", "road_safety"])))
cond_transport_cb_issues_supply_side_tech = (results_all_pp["strategy_code"]=="PFLO:SUPPLY_SIDE_TECH") & (results_all_pp["variable"].apply(lambda x : any(k in x for k in ["congestion", "road_safety"])))

results_all_pp.loc[cond_transport_cb_issues_better_base, "value"] = 0.0
results_all_pp.loc[cond_transport_cb_issues_supply_side_tech, "value"] = 0.0

#in ALL, cut the benefits in half to be on the safe side.
cond_cut_ben_in_half_all_plur =  (results_all_pp["strategy_code"]=="PFLO:ALL_PLUR") & (results_all_pp["variable"].apply(lambda x : any(k in x for k in ["congestion", "road_safety"])))
cond_cut_ben_in_half_all_non_stopping_def_plur =  (results_all_pp["strategy_code"]=='PFLO:ALL_NO_STOPPING_DEFORESTATION_PLUR') & (results_all_pp["variable"].apply(lambda x : any(k in x for k in ["congestion", "road_safety"])))

results_all_pp.loc[cond_cut_ben_in_half_all_plur, "value"] *= 0.5 
results_all_pp.loc[cond_cut_ben_in_half_all_non_stopping_def_plur, "value"] *= 0.5 

#POST PROCESS WASO CB ISSUES
#where moving from incineration to landfilling appears to have benefits
#when this move should probably not occur
cond_waso_cb_issue =  (results_all_pp["strategy_code"]=="PFLO:SUPPLY_SIDE_TECH") & (results_all_pp["variable"].apply(lambda x : any(k in x for k in ["cb:waso:technical_cost:waste_management"])))
results_all_pp.loc[cond_waso_cb_issue, "value"] = 0.0


#SHIFT any stray costs incurred from 2015 to 2025 to 2025 and 2035
results_all_pp_before_shift = results_all_pp.copy() #keep copy of earlier results just in case/for comparison
res_pre2025 = results_all_pp.query(f"time_period<{SSP_GLOBAL_TIME_PERIOD_TX_START}")#get the subset of early costs
res_pre2025["variable"] = res_pre2025["variable"] + "_shifted" + (res_pre2025["time_period"]+SSP_GLOBAL_TIME_PERIOD_0).astype(str)#create a new variable so they can be recognized as shifted costs
res_pre2025["time_period"] = res_pre2025["time_period"]+SSP_GLOBAL_TIME_PERIOD_TX_START #shift the time period

results_all_pp = pd.concat([results_all_pp, res_pre2025], ignore_index = True) #paste the results

results_all_pp.loc[results_all_pp["time_period"]<SSP_GLOBAL_TIME_PERIOD_TX_START,'value'] = 0 #set pre-2025 costs to 0


#Write
CB_OUTPUT_FILENAME = build_path([OUTPUT_PATH, "cost_benefit_results.csv"])
results_all_pp.to_csv(CB_OUTPUT_FILENAME, index = False)

#results_system.to_csv(RESULTS_SYSTEMS_PATH, index = False)
#results_tx.to_csv(RESULTS_TX_SYSTEMS_PATH, index = False)

Resolving Interactions in SCOE :  
No interactions, skipping... TORNADO:AGRC:DEC_CH4_RICE
Resolving Interactions in INEN :  
No interactions, skipping... TORNADO:AGRC:DEC_CH4_RICE
Resolving Interactions in SCOE :  
No interactions, skipping... TORNADO:AGRC:DEC_EXPORTS
Resolving Interactions in INEN :  
No interactions, skipping... TORNADO:AGRC:DEC_EXPORTS
Resolving Interactions in SCOE :  
No interactions, skipping... TORNADO:AGRC:DEC_LOSSES_SUPPLY_CHAIN
Resolving Interactions in INEN :  
No interactions, skipping... TORNADO:AGRC:DEC_LOSSES_SUPPLY_CHAIN
Resolving Interactions in SCOE :  
No interactions, skipping... TORNADO:AGRC:INC_CONSERVATION_AGRICULTURE
Resolving Interactions in INEN :  
No interactions, skipping... TORNADO:AGRC:INC_CONSERVATION_AGRICULTURE
Resolving Interactions in SCOE :  
No interactions, skipping... TORNADO:AGRC:INC_PRODUCTIVITY
Resolving Interactions in INEN :  
No interactions, skipping... TORNADO:AGRC:INC_PRODUCTIVITY
Resolving Interactions in SCOE :  
No in

/tmp/ipykernel_107807/428381665.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_pre2025["variable"] = res_pre2025["variable"] + "_shifted" + (res_pre2025["time_period"]+SSP_GLOBAL_TIME_PERIOD_0).astype(str)#create a new variable so they can be recognized as shifted costs
/tmp/ipykernel_107807/428381665.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_pre2025["time_period"] = res_pre2025["time_period"]+SSP_GLOBAL_TIME_PERIOD_TX_START #shift the time period
